In [39]:
import json
import pickle
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np

In [2]:
with open('/root/llm_quant_safety/data/hotpotqa/hotpot_dev_distractor_v1.json', 'r') as f:
    hotpotqa_json = json.load(f)

In [3]:
len(hotpotqa_json)

7405

In [4]:
hotpotqa_json[0]

{'_id': '5a8b57f25542995d1e6f1371',
 'answer': 'yes',
 'question': 'Were Scott Derrickson and Ed Wood of the same nationality?',
 'supporting_facts': [['Scott Derrickson', 0], ['Ed Wood', 0]],
 'context': [['Ed Wood (film)',
   ['Ed Wood is a 1994 American biographical period comedy-drama film directed and produced by Tim Burton, and starring Johnny Depp as cult filmmaker Ed Wood.',
    " The film concerns the period in Wood's life when he made his best-known films as well as his relationship with actor Bela Lugosi, played by Martin Landau.",
    ' Sarah Jessica Parker, Patricia Arquette, Jeffrey Jones, Lisa Marie, and Bill Murray are among the supporting cast.']],
  ['Scott Derrickson',
   ['Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer.',
    ' He lives in Los Angeles, California.',
    ' He is best known for directing horror films such as "Sinister", "The Exorcism of Emily Rose", and "Deliver Us From Evil", as well as the 2016 Marvel Cinema

In [16]:
def json2string_hotpotqa(single_json):
    lines_context = []
    for title_number, (title, facts) in enumerate(single_json['context']):
        lines_context.append(f'# {title_number+1}. {title}')
        for fact_number, fact in enumerate(facts):
            lines_context.append(f'\t{fact_number+1}. {fact}')
        lines_context.append('\n')
    
    return '\n'.join(lines_context)

In [17]:
example_string = json2string_hotpotqa(hotpotqa_json[0])

In [18]:
print(example_string)

# 1. Ed Wood (film)
	1. Ed Wood is a 1994 American biographical period comedy-drama film directed and produced by Tim Burton, and starring Johnny Depp as cult filmmaker Ed Wood.
	2.  The film concerns the period in Wood's life when he made his best-known films as well as his relationship with actor Bela Lugosi, played by Martin Landau.
	3.  Sarah Jessica Parker, Patricia Arquette, Jeffrey Jones, Lisa Marie, and Bill Murray are among the supporting cast.


# 2. Scott Derrickson
	1. Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer.
	2.  He lives in Los Angeles, California.
	3.  He is best known for directing horror films such as "Sinister", "The Exorcism of Emily Rose", and "Deliver Us From Evil", as well as the 2016 Marvel Cinematic Universe installment, "Doctor Strange."


# 3. Woodson, Arkansas
	1. Woodson is a census-designated place (CDP) in Pulaski County, Arkansas, in the United States.
	2.  Its population was 403 at the 2010 census.
	3.  It

# Pandas

In [20]:
for single_json in hotpotqa_json:
    single_json['context'] = json2string_hotpotqa(single_json)

In [22]:
hotpotqa_df = pd.DataFrame(hotpotqa_json)

In [23]:
hotpotqa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7405 entries, 0 to 7404
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   _id               7405 non-null   object
 1   answer            7405 non-null   object
 2   question          7405 non-null   object
 3   supporting_facts  7405 non-null   object
 4   context           7405 non-null   object
 5   type              7405 non-null   object
 6   level             7405 non-null   object
dtypes: object(7)
memory usage: 405.1+ KB


In [24]:
hotpotqa_df.head()

,_id,answer,question,supporting_facts,context,type,level
0,5a8b57f25542995d1e6f1371,yes,Were Scott Derrickson and Ed Wood of the same ...,"[[Scott Derrickson, 0], [Ed Wood, 0]]",# 1. Ed Wood (film)\n\t1. Ed Wood is a 1994 Am...,comparison,hard
1,5a8c7595554299585d9e36b6,Chief of Protocol,What government position was held by the woman...,"[[Kiss and Tell (1945 film), 0], [Shirley Temp...",# 1. Meet Corliss Archer\n\t1. Meet Corliss Ar...,bridge,hard
2,5a85ea095542994775f606a8,Animorphs,"What science fantasy young adult series, told ...","[[The Hork-Bajir Chronicles, 0], [The Hork-Baj...",# 1. Andre Norton Award\n\t1. The Andre Norton...,bridge,hard
3,5adbf0a255429947ff17385a,no,Are the Laleli Mosque and Esma Sultan Mansion ...,"[[Laleli Mosque, 0], [Esma Sultan Mansion, 0]]",# 1. Esma Sultan (daughter of Abdülaziz)\n\t1....,comparison,hard
4,5a8e3ea95542995a26add48d,"Greenwich Village, New York City","The director of the romantic comedy ""Big Stone...","[[Big Stone Gap (film), 0], [Adriana Trigiani,...",# 1. Just Another Romantic Wrestling Comedy\n\...,bridge,hard


In [26]:
hotpotqa_df.type.value_counts()

type
bridge        5918
comparison    1487
Name: count, dtype: int64

In [27]:
hotpotqa_df.level.value_counts()

level
hard    7405
Name: count, dtype: int64

In [28]:
hotpotqa_df.to_csv('/root/llm_quant_safety/data/hotpotqa/hotpot_dev_distractor_v1.csv', index=False)

In [29]:
hotpotqa_df.sample(50).to_csv('/root/llm_quant_safety/data/hotpotqa/hotpot_dev_distractor_v1_sample_50.csv', index=False)

# Sort by length

In [31]:
import pandas as pd
from transformers import AutoTokenizer

In [34]:
PROMPT_TEMPLATE = """
You are helpful AI assistant.
Briefly answer the question based on provided info.

# Information
{INFO}

# Question
{QUESTION}
""".strip()

In [33]:
df = pd.read_csv('/root/llm_quant_safety/data/hotpotqa/hotpot_dev_distractor_v1.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7405 entries, 0 to 7404
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   _id               7405 non-null   object
 1   answer            7405 non-null   object
 2   question          7405 non-null   object
 3   supporting_facts  7405 non-null   object
 4   context           7405 non-null   object
 5   type              7405 non-null   object
 6   level             7405 non-null   object
dtypes: object(7)
memory usage: 405.1+ KB


In [37]:
prompts = [
    PROMPT_TEMPLATE.format(
        INFO=row["context"], 
        QUESTION=row["question"], 
    ) for idx, row in df.iterrows()
]

In [47]:
import pandas as pd
import numpy as np

# Предполагаем, что df_sort - это ваш отсортированный датафрейм

def split_dataset(df, column='tokenized_len', parts=3):
    total_sum = df[column].sum()
    target_sum = total_sum / parts
    
    split_indices = []
    current_sum = 0
    current_part = 0
    
    for i, value in enumerate(df[column]):
        current_sum += value
        if current_sum >= target_sum and current_part < parts - 1:
            split_indices.append(i)
            current_sum = 0
            current_part += 1
    
    dataset_parts = []
    start = 0
    for end in split_indices:
        dataset_parts.append(df.iloc[start:end])
        start = end
    dataset_parts.append(df.iloc[start:]) 
    
    return dataset_parts

## Llama

In [32]:
# model_name = "kharinaev/Llama-3.1-8B-Instruct-quip"
# model_name = '/root/llm_quant_safety/quantization/models/Mistral-7B-Instruct-v0.2-quip-2bit'
model_name = '/root/llm_quant_safety/quantization/models/Llama-3.1-8B-Instruct-quip-2bit/'
tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.add_special_tokens({'pad_token': '<|eot_id|>'}) # LLAMA
tokenizer.pad_token = tokenizer.eos_token

In [40]:
tokenized = [
    tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}], 
        tokenize=True, 
        add_generation_prompt=True
    ) for prompt in tqdm(prompts)
]

  0%|          | 0/7405 [00:00<?, ?it/s]

In [42]:
df['tokenized_len'] = [len(t) for t in tokenized]
df['tokenized_len'].describe()

count    7405.000000
mean     1557.559622
std       376.207322
min       135.000000
25%      1316.000000
50%      1527.000000
75%      1767.000000
max      4057.000000
Name: tokenized_len, dtype: float64

In [43]:
df_sort = df.sort_values('tokenized_len')

In [44]:
sum_token_len = df_sort.tokenized_len.sum()
sum_token_len

11533729

In [45]:
sum_token_len // 3

3844576

In [48]:
parts = split_dataset(df_sort)

for i, part in enumerate(parts):
    print(f"Part {i+1}:")
    print(f"  Rows: {len(part)}")
    print(f"  Sum of tokenized_len: {part['tokenized_len'].sum()}")
    print()

Part 1:
  Rows: 3120
  Sum of tokenized_len: 3843586

Part 2:
  Rows: 2399
  Sum of tokenized_len: 3845688

Part 3:
  Rows: 1886
  Sum of tokenized_len: 3844455



In [49]:
for i in range(3):
    parts[i].sort_values('tokenized_len').to_csv(f'/root/llm_quant_safety/data/hotpotqa/parts/llama_p{i}.csv', index=False)

In [50]:
for i in range(3):
    display(pd.read_csv(f'/root/llm_quant_safety/data/hotpotqa/parts/llama_p{i}.csv').info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3120 entries, 0 to 3119
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   _id               3120 non-null   object
 1   answer            3120 non-null   object
 2   question          3120 non-null   object
 3   supporting_facts  3120 non-null   object
 4   context           3120 non-null   object
 5   type              3120 non-null   object
 6   level             3120 non-null   object
 7   tokenized_len     3120 non-null   int64 
dtypes: int64(1), object(7)
memory usage: 195.1+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2399 entries, 0 to 2398
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   _id               2399 non-null   object
 1   answer            2399 non-null   object
 2   question          2399 non-null   object
 3   supporting_facts  2399 non-null   object
 4   context           2399 non-null   object
 5   type              2399 non-null   object
 6   level             2399 non-null   object
 7   tokenized_len     2399 non-null   int64 
dtypes: int64(1), object(7)
memory usage: 150.1+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1886 entries, 0 to 1885
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   _id               1886 non-null   object
 1   answer            1886 non-null   object
 2   question          1886 non-null   object
 3   supporting_facts  1886 non-null   object
 4   context           1886 non-null   object
 5   type              1886 non-null   object
 6   level             1886 non-null   object
 7   tokenized_len     1886 non-null   int64 
dtypes: int64(1), object(7)
memory usage: 118.0+ KB


None

## Mistral

In [51]:
model_name = '/root/llm_quant_safety/quantization/models/Mistral-7B-Instruct-v0.2-quip-2bit'
# model_name = '/root/llm_quant_safety/quantization/models/Llama-3.1-8B-Instruct-quip-2bit/'
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [52]:
tokenized = [
    tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}], 
        tokenize=True, 
        add_generation_prompt=True
    ) for prompt in tqdm(prompts)
]

  0%|          | 0/7405 [00:00<?, ?it/s]

In [53]:
df['tokenized_len'] = [len(t) for t in tokenized]
df['tokenized_len'].describe()

count    7405.000000
mean     1821.815260
std       437.435342
min       153.000000
25%      1538.000000
50%      1786.000000
75%      2068.000000
max      4741.000000
Name: tokenized_len, dtype: float64

In [54]:
df_sort = df.sort_values('tokenized_len')

In [55]:
sum_token_len = df_sort.tokenized_len.sum()
sum_token_len

13490542

In [56]:
sum_token_len // 3

4496847

In [57]:
parts = split_dataset(df_sort)

for i, part in enumerate(parts):
    print(f"Part {i+1}:")
    print(f"  Rows: {len(part)}")
    print(f"  Sum of tokenized_len: {part['tokenized_len'].sum()}")
    print()

Part 1:
  Rows: 3115
  Sum of tokenized_len: 4495709

Part 2:
  Rows: 2401
  Sum of tokenized_len: 4498413

Part 3:
  Rows: 1889
  Sum of tokenized_len: 4496420



In [58]:
for i in range(3):
    parts[i].sort_values('tokenized_len').to_csv(f'/root/llm_quant_safety/data/hotpotqa/parts/mistral_p{i}.csv', index=False)

In [59]:
for i in range(3):
    display(pd.read_csv(f'/root/llm_quant_safety/data/hotpotqa/parts/mistral_p{i}.csv').info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3115 entries, 0 to 3114
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   _id               3115 non-null   object
 1   answer            3115 non-null   object
 2   question          3115 non-null   object
 3   supporting_facts  3115 non-null   object
 4   context           3115 non-null   object
 5   type              3115 non-null   object
 6   level             3115 non-null   object
 7   tokenized_len     3115 non-null   int64 
dtypes: int64(1), object(7)
memory usage: 194.8+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2401 entries, 0 to 2400
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   _id               2401 non-null   object
 1   answer            2401 non-null   object
 2   question          2401 non-null   object
 3   supporting_facts  2401 non-null   object
 4   context           2401 non-null   object
 5   type              2401 non-null   object
 6   level             2401 non-null   object
 7   tokenized_len     2401 non-null   int64 
dtypes: int64(1), object(7)
memory usage: 150.2+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   _id               1889 non-null   object
 1   answer            1889 non-null   object
 2   question          1889 non-null   object
 3   supporting_facts  1889 non-null   object
 4   context           1889 non-null   object
 5   type              1889 non-null   object
 6   level             1889 non-null   object
 7   tokenized_len     1889 non-null   int64 
dtypes: int64(1), object(7)
memory usage: 118.2+ KB


None

# Fix CUDA out of memory for last examples

In [60]:
df_p2 = pd.read_csv('/root/llm_quant_safety/data/hotpotqa/responses/parts/llama_quip2_p2.csv')
df_p2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1886 entries, 0 to 1885
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   _id               1886 non-null   object
 1   answer            1886 non-null   object
 2   question          1886 non-null   object
 3   supporting_facts  1886 non-null   object
 4   context           1886 non-null   object
 5   type              1886 non-null   object
 6   level             1886 non-null   object
 7   tokenized_len     1886 non-null   int64 
 8   llm_responses     1872 non-null   object
dtypes: int64(1), object(8)
memory usage: 132.7+ KB


In [62]:
df_p2_undone = df_p2.iloc[1872:]

In [63]:
df_p2_undone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 1872 to 1885
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   _id               14 non-null     object
 1   answer            14 non-null     object
 2   question          14 non-null     object
 3   supporting_facts  14 non-null     object
 4   context           14 non-null     object
 5   type              14 non-null     object
 6   level             14 non-null     object
 7   tokenized_len     14 non-null     int64 
 8   llm_responses     0 non-null      object
dtypes: int64(1), object(8)
memory usage: 1.1+ KB


In [65]:
df_p2_undone.to_csv('/root/llm_quant_safety/data/hotpotqa/responses/parts/llama_quip2_p3.csv', index=False)

In [66]:
df.iloc[:1872].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1872 entries, 0 to 1871
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   _id               1872 non-null   object
 1   answer            1872 non-null   object
 2   question          1872 non-null   object
 3   supporting_facts  1872 non-null   object
 4   context           1872 non-null   object
 5   type              1872 non-null   object
 6   level             1872 non-null   object
 7   tokenized_len     1872 non-null   int64 
dtypes: int64(1), object(7)
memory usage: 117.1+ KB


In [67]:
df.iloc[:1872].to_csv('/root/llm_quant_safety/data/hotpotqa/responses/parts/llama_quip2_p2.csv', index=False)

# Add jsons for hallu eval

In [68]:
with open('/root/llm_quant_safety/data/hotpotqa/hotpot_dev_distractor_v1.json', 'r') as f:
    hotpotqa_json = json.load(f)

In [69]:
len(hotpotqa_json)

7405

In [70]:
hotpotqa_json[0]

{'_id': '5a8b57f25542995d1e6f1371',
 'answer': 'yes',
 'question': 'Were Scott Derrickson and Ed Wood of the same nationality?',
 'supporting_facts': [['Scott Derrickson', 0], ['Ed Wood', 0]],
 'context': [['Ed Wood (film)',
   ['Ed Wood is a 1994 American biographical period comedy-drama film directed and produced by Tim Burton, and starring Johnny Depp as cult filmmaker Ed Wood.',
    " The film concerns the period in Wood's life when he made his best-known films as well as his relationship with actor Bela Lugosi, played by Martin Landau.",
    ' Sarah Jessica Parker, Patricia Arquette, Jeffrey Jones, Lisa Marie, and Bill Murray are among the supporting cast.']],
  ['Scott Derrickson',
   ['Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer.',
    ' He lives in Los Angeles, California.',
    ' He is best known for directing horror films such as "Sinister", "The Exorcism of Emily Rose", and "Deliver Us From Evil", as well as the 2016 Marvel Cinema

In [77]:
hotpotqa_json[0]['context'][0]

['Ed Wood (film)',
 ['Ed Wood is a 1994 American biographical period comedy-drama film directed and produced by Tim Burton, and starring Johnny Depp as cult filmmaker Ed Wood.',
  " The film concerns the period in Wood's life when he made his best-known films as well as his relationship with actor Bela Lugosi, played by Martin Landau.",
  ' Sarah Jessica Parker, Patricia Arquette, Jeffrey Jones, Lisa Marie, and Bill Murray are among the supporting cast.']]

In [73]:
id2json = {}

In [78]:
for record in hotpotqa_json:
    id2json[record['_id']] = record['context']

In [71]:
df_abl = pd.read_csv('/root/llm_quant_safety/data/hotpotqa/responses/llama_abl.csv')
df_abl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7405 entries, 0 to 7404
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   _id               7405 non-null   object
 1   answer            7405 non-null   object
 2   question          7405 non-null   object
 3   supporting_facts  7405 non-null   object
 4   context           7405 non-null   object
 5   type              7405 non-null   object
 6   level             7405 non-null   object
 7   llm_responses     7405 non-null   object
dtypes: object(8)
memory usage: 462.9+ KB


In [72]:
df_abl.head()

,_id,answer,question,supporting_facts,context,type,level,llm_responses
0,5a8b57f25542995d1e6f1371,yes,Were Scott Derrickson and Ed Wood of the same ...,"[['Scott Derrickson', 0], ['Ed Wood', 0]]",# 1. Ed Wood (film)\n\t1. Ed Wood is a 1994 Am...,comparison,hard,"No, Scott Derrickson and Ed Wood were not of t..."
1,5a8c7595554299585d9e36b6,Chief of Protocol,What government position was held by the woman...,"[['Kiss and Tell (1945 film)', 0], ['Shirley T...",# 1. Meet Corliss Archer\n\t1. Meet Corliss Ar...,bridge,hard,"Unfortunately, the information provided does n..."
2,5a85ea095542994775f606a8,Animorphs,"What science fantasy young adult series, told ...","[['The Hork-Bajir Chronicles', 0], ['The Hork-...",# 1. Andre Norton Award\n\t1. The Andre Norton...,bridge,hard,"Based on the provided information, I can answe..."
3,5adbf0a255429947ff17385a,no,Are the Laleli Mosque and Esma Sultan Mansion ...,"[['Laleli Mosque', 0], ['Esma Sultan Mansion',...",# 1. Esma Sultan (daughter of Abdülaziz)\n\t1....,comparison,hard,I can help you with that!\n\nAccording to the ...
4,5a8e3ea95542995a26add48d,"Greenwich Village, New York City","The director of the romantic comedy ""Big Stone...","[['Big Stone Gap (film)', 0], ['Adriana Trigia...",# 1. Just Another Romantic Wrestling Comedy\n\...,bridge,hard,"Based on the information provided, the directo..."


In [79]:
df_abl['context_lists'] = df_abl._id.map(id2json)

In [80]:
df_abl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7405 entries, 0 to 7404
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   _id               7405 non-null   object
 1   answer            7405 non-null   object
 2   question          7405 non-null   object
 3   supporting_facts  7405 non-null   object
 4   context           7405 non-null   object
 5   type              7405 non-null   object
 6   level             7405 non-null   object
 7   llm_responses     7405 non-null   object
 8   context_lists     7405 non-null   object
dtypes: object(9)
memory usage: 520.8+ KB


In [81]:
df_abl.head()

,_id,answer,question,supporting_facts,context,type,level,llm_responses,context_lists
0,5a8b57f25542995d1e6f1371,yes,Were Scott Derrickson and Ed Wood of the same ...,"[['Scott Derrickson', 0], ['Ed Wood', 0]]",# 1. Ed Wood (film)\n\t1. Ed Wood is a 1994 Am...,comparison,hard,"No, Scott Derrickson and Ed Wood were not of t...","[[Ed Wood (film), [Ed Wood is a 1994 American ..."
1,5a8c7595554299585d9e36b6,Chief of Protocol,What government position was held by the woman...,"[['Kiss and Tell (1945 film)', 0], ['Shirley T...",# 1. Meet Corliss Archer\n\t1. Meet Corliss Ar...,bridge,hard,"Unfortunately, the information provided does n...","[[Meet Corliss Archer, [Meet Corliss Archer, a..."
2,5a85ea095542994775f606a8,Animorphs,"What science fantasy young adult series, told ...","[['The Hork-Bajir Chronicles', 0], ['The Hork-...",# 1. Andre Norton Award\n\t1. The Andre Norton...,bridge,hard,"Based on the provided information, I can answe...","[[Andre Norton Award, [The Andre Norton Award ..."
3,5adbf0a255429947ff17385a,no,Are the Laleli Mosque and Esma Sultan Mansion ...,"[['Laleli Mosque', 0], ['Esma Sultan Mansion',...",# 1. Esma Sultan (daughter of Abdülaziz)\n\t1....,comparison,hard,I can help you with that!\n\nAccording to the ...,"[[Esma Sultan (daughter of Abdülaziz), [Esma S..."
4,5a8e3ea95542995a26add48d,"Greenwich Village, New York City","The director of the romantic comedy ""Big Stone...","[['Big Stone Gap (film)', 0], ['Adriana Trigia...",# 1. Just Another Romantic Wrestling Comedy\n\...,bridge,hard,"Based on the information provided, the directo...","[[Just Another Romantic Wrestling Comedy, [Jus..."


In [82]:
df_abl.to_csv('/root/llm_quant_safety/data/hotpotqa/responses/llama_abl_with_context_lists.csv', index=False)